# Estimation of the Vs30 (m/s) with the mHVSR using low-dimensional models.

## Information
This file is a free software as a part of:
>{citation}.

Copyright (C) 2025 Kushal Sharma Wagle (wagleyk@vt.edu).


## Citation

If you use _hvsrpy_ in your research or consulting, we ask you please cite the following:

>Joseph Vantassel. (2020). jpvantassel/hvsrpy: latest (Concept). Zenodo.
[http://doi.org/10.5281/zenodo.3666956](http://doi.org/10.5281/zenodo.3666956)


>Vantassel, J.P. (2025). hvsrpy: An Open‐Source Python Package for Microtremor and Earthquake Horizontal‐to‐Vertical Spectral Ratio Processing. Seismological Research Letters 2025. [https://doi.org/10.1785/0220240395].

_Note: For software, version specific citations should be preferred to
general concept citations, such as that listed above. To generate a version
specific citation for hvsrpy, please use the citation tool on the hvsrpy
[archive](http://doi.org/10.5281/zenodo.3666956)._

## About this notebook

This notebook illustrates how mHVSR can be leveraged to estimate the Vs30 for the site. The processing has been done following the SESAME (2004) guidelines.
If you use this notebook, please also cite SESAME (2004) to recognize their original work.

> SESAME (2004). Guidelines for the Implementation of the H/V Spectral Ratio Technique on Ambient Vibrations
> Measurements, Processing, and Interpretation. European Commission - Research General Directorate, 62,
> European Commission - Research General Directorate.

To use this notebook, you need at least one clear resonance peak in the mHVSR curve. For more information regarding the clear resonance peak, see the SESAME (2004) guidelines. Also, with this notebook, if the clarity criteria FAILS, it is not recommended to proceed further with the prediction.


Additionally, you need topographic features: elevation and average elevation around 1500m diameter of the station. These shall be computed using 1 arcsec Digital Elevation Model (DEM) to remain consistent with the model development. If you don't have these data, you can use another notebook and use the model "Single mode ANN" with just the microtremor recording.

Steps to use:
1. Simply, download the zipped folder with the name "Models Public", and do not change the files structure in it.
2. Input all the required parameters, input recording (full or relative path is fine) and topographic features in the input section.
3. Restart Kernel and run all cells.

In [ ]:
import numpy as np
import pandas as pd
import os
import hvsrpy
from hvsrpy import sesame
import obspy
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
from scipy.stats import skew
import joblib

plt.style.use(hvsrpy.postprocessing.HVSRPY_MPL_STYLE)

### Data Input and Recording Check

In [ ]:
# Data Input
# input_recording = "CI.FUR-Rec288-Sen450.mseed"  #Actual elevation = -44m and elevation_1500m_avg = -43.80603662m.
input_recording = "AR.STN01.A1.2019.006"  #Actual elevation = 2246m and elevation_1500m_avg = 2237.903656m.
# input_recording = "UT.STN09_20130320_020000.miniseed" #Actual elevation = 35m and elevation_1500m_avg = 35.33800078m.
elevation =  2246 #in meters.
elevation_1500m_avg =  2237.903656 # in meters.


stream = obspy.read(input_recording)

# Check trace count
if len(stream) != 3:
    print("Recording should have exactly three traces.")
else:
    # Find latest start time and earliest end time
    start_times = [tr.stats.starttime for tr in stream]
    end_times = [tr.stats.endtime for tr in stream]

    common_start = max(start_times)
    common_end = min(end_times)

    # Check duration
    common_duration = common_end - common_start
    if common_duration <= 0:
        print("No overlapping time window between traces.")
    else:
        # Trim all traces to common time window
        stream.trim(starttime=common_start, endtime=common_end)

        # Save to MiniSEED file
        stream.write(f"updated_{input_recording}", format="MSEED")

        # Check if duration exceeds 30 minutes
        if common_duration >= 1800:  # 1800 seconds = 30 minutes
            print(f"Trimmed duration is {common_duration/60 :.2f} minutes.")
        else:
            print("Trimmed duration is less than 30 minutes.")

In [ ]:
# HVSR Preprocessing Settings
preprocessing_settings = hvsrpy.settings.HvsrPreProcessingSettings()
preprocessing_settings.detrend = "linear"
significant_cycles = 15  # require 15 significant cycles.
time_windows = 35  # require 35 time windows.
duration_in_seconds = common_duration  # window length (s)
windowlength_in_seconds = duration_in_seconds / time_windows
preprocessing_settings.window_length_in_seconds = windowlength_in_seconds

print("Preprocessing Summary")
print("-"*60)
preprocessing_settings.psummary()

# HVSR Processing Settings
processing_settings = hvsrpy.settings.HvsrTraditionalProcessingSettings()
processing_settings.window_type_and_width = ("tukey", 0.2)
processing_settings.smoothing=dict(operator="konno_and_ohmachi",
                                   bandwidth=40,
                                   center_frequencies_in_hz=np.geomspace(0.05, 50, 256))
processing_settings.method_to_combine_horizontals = "geometric_mean"
processing_settings.handle_dissimilar_time_steps_by = "frequency_domain_resampling"

desired_frequency_vector_in_hz = np.geomspace(0.05, 50, 256)
minimum_frequency = significant_cycles / windowlength_in_seconds
fids = desired_frequency_vector_in_hz > minimum_frequency
frequency_resampling_in_hz = desired_frequency_vector_in_hz[fids]
processing_settings.smoothing["center_frequencies_in_hz"] = frequency_resampling_in_hz

print("Processing Summary")
print("-"*60)
processing_settings.psummary()

### HVSR Processing and Manual Window Rejection

In [ ]:
# Compute HVSR

srecords = hvsrpy.read([f"updated_{input_recording}"])
srecords = hvsrpy.preprocess(srecords, preprocessing_settings)
hvsr = hvsrpy.process(srecords, processing_settings)

In [ ]:
# Create HvsrTraditional object
mhvsr = hvsrpy.HvsrTraditional(frequency=hvsr.frequency, amplitude=hvsr.amplitude)

# Perform manual window rejection
hvsrpy.window_rejection.manual_window_rejection(
    mhvsr, y_limit=15, plot_frequency_std=False, fig=None, ax=None  #Change y_limit as required in the plot.
)
plt.close("all")

In [ ]:
search_range_in_hz = (None, None)
verbose = 1

print("\nSESAME (2004) Clarity and Reliability Criteria:")
print("-"*47)
hvsrpy.sesame.reliability(
    windowlength=preprocessing_settings.window_length_in_seconds,
    passing_window_count=np.sum(mhvsr.valid_window_boolean_mask),
    frequency=mhvsr.frequency,
    mean_curve=mhvsr.mean_curve(distribution="lognormal"),
    std_curve=mhvsr.std_curve(distribution="lognormal"),
    search_range_in_hz=search_range_in_hz,
    verbose=verbose,
)
hvsrpy.sesame.clarity(
    frequency=mhvsr.frequency,
    mean_curve=mhvsr.mean_curve(distribution="lognormal"),
    std_curve=mhvsr.std_curve(distribution="lognormal"),
    fn_std=mhvsr.std_fn_frequency(distribution="normal"),
    search_range_in_hz=search_range_in_hz,
    verbose=verbose,
)

fig, axs = hvsrpy.plot_single_panel_hvsr_curves(mhvsr)
plt.show()

### Parameters for low dim model.

In [ ]:
fn_mean = mhvsr.mean_fn_frequency(distribution="lognormal")
an_mean = mhvsr.mean_fn_amplitude(distribution="lognormal")
elevation = elevation
TPI = elevation - elevation_1500m_avg

# Function to compute skewness ignoring leading NaNs and filtering to 0.2–50 Hz
def compute_skew_ignore_leading_nans(freqs, amps):
    # Filter by frequency range
    mask = (freqs > 0.199) & (freqs < 50)
    filtered_amps = amps[mask]

    # Ignore leading NaNs
    first_valid_index = np.argmax(~np.isnan(filtered_amps))
    trimmed = filtered_amps[first_valid_index:]
    valid = trimmed[~np.isnan(trimmed)]

    return skew(valid, bias=False) if len(valid) >= 3 else np.nan


mhvsr_mean_curve = mhvsr.mean_curve(distribution="lognormal")  #Taking the lognormal mean curve among the accepted windows.
skewness = compute_skew_ignore_leading_nans(mhvsr.frequency, mhvsr_mean_curve)

# Assemble into DataFrame
X = pd.DataFrame([{
    "fn_mean": fn_mean,
    "an_mean": an_mean,
    "TPI": TPI,
    "Skewness": skewness,
    "elevation": elevation
}])

# Elevation Binning
elevation_bins = [-500, 0, 500, 1000, 1500, 2000, 2500, 3000]
elevation_labels = [
    'Elevation_Bin_[-500.0, 0.0)',
    'Elevation_Bin_[0.0, 500.0)',
    'Elevation_Bin_[500.0, 1000.0)',
    'Elevation_Bin_[1000.0, 1500.0)',
    'Elevation_Bin_[1500.0, 2000.0)',
    'Elevation_Bin_[2000.0, 2500.0)',
    'Elevation_Bin_[2500.0, 3000.0)'
]

# Bin the elevation into categories
X = X.copy()
X['Elevation_Bin'] = pd.cut(X['elevation'], bins=elevation_bins, labels=elevation_labels, right=False)

# One-hot encode the elevation bin
elevation_dummies = pd.get_dummies(X['Elevation_Bin']).astype(int)

# Concatenate back to original DataFrame
X = pd.concat([X, elevation_dummies], axis=1)

# Optional: Drop 'Elevation_Bin' column if you only need one-hot encoded version
X = X.drop(columns=['elevation','Elevation_Bin'])

In [ ]:
X

### Load low-dimensional models

In [ ]:
# Load Models

linear_model = joblib.load("../Models Public/Models/linear_model.joblib")
decisiontree_model = joblib.load("../Models Public/Models/decision_tree_model.joblib")
randomforest_model = joblib.load("../Models Public/Models/random_forest_model.joblib")
xgboost_model = joblib.load("../Models Public/Models/xgboost_model.joblib")

# Feature Scaling for linear model
X_linear = X.copy().to_numpy()

boxcox_scaler = joblib.load("../Models Public/Models/linear_f0hvsr_scaler.pkl") # fn_mean: Box-Cox Transform
X_linear[:, 0] = boxcox_scaler.transform(X_linear[:, 0].reshape(-1, 1)).flatten()

X_linear[:, 1] = np.log(X_linear[:, 1]) #an_mean: log transformation

std_scaler_tpi = joblib.load("../Models Public/Models/linear_topo_scaler.pkl") #TPI: Standard transformation
X_linear[:, 2] = std_scaler_tpi.transform(X_linear[:, 2].reshape(-1, 1)).flatten()

std_scaler_skew = joblib.load("../Models Public/Models/linear_skewness_scaler.pkl") #Skewness: standard transformation
X_linear[:, 3] = std_scaler_tpi.transform(X_linear[:, 3].reshape(-1, 1)).flatten()

### Predictions

In [ ]:
Vs30_pred_linear = linear_model.predict(X_linear)
Vs30_pred_decisiontree = decisiontree_model.predict(X.to_numpy())
Vs30_pred_randomforest = randomforest_model.predict(X.to_numpy())
Vs30_pred_xgboost = xgboost_model.predict(X.to_numpy())

In [ ]:
# Stack predictions and model labels
models = ["Linear", "Decision Tree", "Random Forest", "XGBoost"]
predictions = [
    np.exp(Vs30_pred_linear),
    np.exp(Vs30_pred_decisiontree),
    np.exp(Vs30_pred_randomforest),
    np.exp(Vs30_pred_xgboost)
]

# Create dataframe
# Create dataframe with 0 decimal precision
vs30_modelwise_df = pd.DataFrame({
    "Model": np.repeat(models, [len(p) for p in predictions]),
    "Vs30 (m/s)": np.round(np.concatenate(predictions), 0).astype(int)
})

#Show Results
vs30_modelwise_df